In [1]:
from Imports import *

In [2]:
from s08_Training import(
    ds_test,
    tokenized_test,
)

Map:   0%|          | 0/189155 [00:00<?, ? examples/s]

Map:   0%|          | 0/15763 [00:00<?, ? examples/s]

Map:   0%|          | 0/5255 [00:00<?, ? examples/s]

Filter:   0%|          | 0/189155 [00:00<?, ? examples/s]

Filter:   0%|          | 0/15763 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/6731 [00:00<?, ? examples/s]

Map:   0%|          | 0/5255 [00:00<?, ? examples/s]

Model found at /home/ludo/Public/Projets_IA/French-English_summary_generation/model

No training needed.

If you want to train a new model, please delete this folder content before launching.

In [3]:
def load_model_and_tokenizer(
    model_path: str | Path=Path("../model"),
) -> tuple[MarianMTModel, MarianTokenizer]:
    """
    Load the model and tokenizer from `model_path` location
    """

    print("Loading model and tokenizer...")
    model = MarianMTModel.from_pretrained(model_path)
    tokenizer = MarianTokenizer.from_pretrained(model_path)
    print("Model and tokenizer loaded successfully!")

    return (model, tokenizer)

In [4]:
def evaluate_translations(
    predictions: list,
    references: list,
) -> dict:
    """
    Evaluate translations using multiple metrics
    """
    # Load evaluation metrics
    bleu = evaluate.load("sacrebleu")
    meteor = evaluate.load("meteor")
    chrf = evaluate.load("chrf")

    # Compute scores
    results = {}

    # BLEU
    bleu_results = bleu.compute(predictions=predictions, references=references)
    results['bleu'] = {
        'score': bleu_results['score'],
        'precisions': bleu_results['precisions']
    }

    # METEOR
    meteor_results = meteor.compute(predictions=predictions, references=references)
    results['meteor'] = meteor_results['meteor']

    # chrF
    chrf_results = chrf.compute(predictions=predictions, references=references)
    results['chrf'] = chrf_results['score']

    return results

In [5]:
def generate_predictions(
    model: MarianMTModel,
    tokenizer: MarianTokenizer,
    eval_dataloader: DataLoader,
    device: torch.device,
) -> list:
    """
    Generate predictions for the test set
    """
    model.eval()
    all_preds = []
    required_keys = {"input_ids", "attention_mask"}

    print("Generating predictions...")
    total_batches = len(eval_dataloader)

    for i, batch in enumerate(eval_dataloader, 1):
        if i % 10 == 0:  # Show progress every 10 batches
            print(f"Processing batch {i}/{total_batches}")

        # Prepare the batch
        numeric_batch = {}
        for k, v in batch.items():
            if k in required_keys:
                tensor_v = torch.tensor(v).to(device)
                if tensor_v.dim() == 1:
                    tensor_v = tensor_v.unsqueeze(0)
                numeric_batch[k] = tensor_v

        # Generate translations
        with torch.no_grad():
            generated_ids = model.generate(
                input_ids=numeric_batch["input_ids"],
                attention_mask=numeric_batch["attention_mask"],
                max_length=128,
                num_beams=4,
                early_stopping=True
            )

        # Decode the predictions
        preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        all_preds.extend(preds)
        torch.cuda.empty_cache()

    print("Prediction generation completed!")
    return all_preds

In [6]:
##########################################

In [7]:
model, tokenizer = load_model_and_tokenizer()

Loading model and tokenizer...

Model and tokenizer loaded successfully!

In [8]:
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = model.to(device)

Using device: cuda

In [9]:
# Prepare the DataLoader
eval_dataloader = DataLoader(tokenized_test, batch_size=1, shuffle=False)

In [10]:
 # Generate predictions
predictions = generate_predictions(model, tokenizer, eval_dataloader, device)

Generating predictions...

Processing batch 10/5255

Processing batch 20/5255

Processing batch 30/5255

Processing batch 40/5255

...

Processing batch 5220/5255

Processing batch 5230/5255

Processing batch 5240/5255

Processing batch 5250/5255

Prediction generation completed!

In [11]:
import evaluate
# Prepare references
references = [[ex["translation"]["fr"]] for ex in ds_test]

# Evaluate translations
results = evaluate_translations(predictions, references)

[nltk_data] Downloading package wordnet to /home/ludo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/ludo/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/ludo/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [13]:
# Display results
print("\nEvaluation results:")
print(f"BLEU score: {results['bleu']['score']:.2f}")
print(f"METEOR score: {results['meteor']:.2f}")
print(f"chrF score: {results['chrf']:.2f}")

Evaluation results:

BLEU score: 37.63

METEOR score: 0.52

chrF score: 60.55

In [14]:
# Show detailed BLEU precisions
print("\nDetailed BLEU precisions:")
for i, precision in enumerate(results['bleu']['precisions'], 1):
    print(f"{i}-gram precision: {precision:.2f}")

Detailed BLEU precisions:

1-gram precision: 66.93

2-gram precision: 47.34

3-gram precision: 34.96

4-gram precision: 25.99

In [15]:
# Show sample translations
print("\nSample translations (first 5):")
for i in range(min(5, len(predictions))):
    print(f"\nExample {i+1}:")
    print(f"Prediction: {predictions[i]}")
    print(f"Reference: {references[i][0]}")

Sample translations (first 5):

Example 1:

Prediction: :

Reference: :

Example 2:

...

Prediction: http: / /localhost:631/ documentation. html

Reference: http: / /localhost:631/ documentation. html

Example 5:

Prediction: description

Reference: arnold kraschinski